### test 2

#### Lưu ý
- trong spark streaming dùng readStream để xử lý dữ liệu
- và dùng writeStreaming để ghi dữ liệu ra màn hình hoặc lưu dữ liệu


In [1]:
import numpy as np
from pyspark.sql import SparkSession

# Spark session & context
spark = (SparkSession
         .builder
         .master('local')
         .appName('kafka-streaming')
         .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.5")
         .getOrCreate())
sc = spark.sparkContext

# Khởi tạo Kafka consumer
kafka_bootstrap_servers = 'kafka1:19091,kafka2:19092,kafka3:19093' # Địa chỉ Kafka của các container Kafka
kafka_topic = 'nguyentri'  # Tên topic Kafka

# Subscribe to 1 topic
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
  .option("subscribe", kafka_topic) \
  .load()

In [2]:
# # Chuyển đổi cột giá trị thành chuỗi
#df = df.selectExpr("CAST(value AS )")
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [3]:
# chạy được
# Decode the value column using UTF-8 encoding
decoded_df = df.withColumn("decoded_value", df["value"].cast("string").alias("decoded_value"))
# Parse the JSON data in the decoded_value column
parsed_df = decoded_df.select("decoded_value")

# Start the streaming query
query = parsed_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()
query.awaitTermination()

KeyboardInterrupt: 

In [ ]:
# chạy được
# Decode the value column using UTF-8 encoding
decoded_df = df.withColumn("decoded_value", df["value"].cast("string").alias("decoded_value"))
# Parse the JSON data in the decoded_value column
parsed_df = decoded_df.selectExpr("CAST(decoded_value AS STRING)")
# Lấy giá trị của cột "value"
values = decoded_df.select("decoded_value")

# In giá trị ra màn hình
query = (values.writeStream
         .outputMode("append")
         .format("console")
         .start())

# Chờ cho quá trình stream kết thúc
query.awaitTermination()

In [8]:
# # Lấy giá trị của cột "value"
# values = df.select("value")

# # Lưu kết quả vào tệp CSV
# query = (values.writeStream
#          .outputMode("append")
#          .format("csv")
#          .option("checkpointLocation", "./checkpoint")
#          .option("path", "./output")
#          .start())

# # Chờ cho quá trình stream kết thúc
# query.awaitTermination()

In [15]:
# def print_row(row):
#     print(row)
# writer = df.writeStream.foreach(print_row)

In [11]:
# Định nghĩa schema cho dữ liệu JSON
schema = StructType([
    StructField("data", ArrayType(ArrayType(FloatType())), nullable=False)
])

In [19]:

from pyspark.sql.functions import from_json, udf
from pyspark.sql.types import StructType, StructField, FloatType, ArrayType
# Decode cột value bằng UTF-8 encoding
decoded_df = df.withColumn("decoded_value", df["value"].cast("string").alias("decoded_value"))

# Parse dữ liệu JSON trong cột decoded_value
parsed_df = decoded_df.select(from_json(decoded_df.decoded_value, schema).alias("data"))

# Hàm UDF để chuyển đổi dữ liệu thành numpy array
@udf(returnType=ArrayType(FloatType()))
def to_numpy(data):
    return np.array(data).reshape(-1)

# Chuyển đổi dữ liệu thành numpy array và in ra console
parsed_df = parsed_df.withColumn("numpy_data", to_numpy(parsed_df.data))

# In dữ liệu numpy ra màn hình console
query = parsed_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

# Chờ cho quá trình stream kết thúc
query.awaitTermination()

StreamingQueryException: 'Writing job aborted.\n=== Streaming Query ===\nIdentifier: [id = 36cd3c2c-5a23-4cf3-9d7d-7574780d8da7, runId = a27c0b17-a2f9-47de-8bf7-baf581e89c8b]\nCurrent Committed Offsets: {KafkaV2[Subscribe[trading4]]: {"trading4":{"0":108}}}\nCurrent Available Offsets: {KafkaV2[Subscribe[trading4]]: {"trading4":{"0":109}}}\n\nCurrent State: ACTIVE\nThread State: RUNNABLE\n\nLogical Plan:\nProject [data#466, to_numpy(data#466) AS numpy_data#469]\n+- Project [jsontostructs(StructField(data,ArrayType(ArrayType(FloatType,true),true),false), decoded_value#457, Some(Etc/UTC)) AS data#466]\n   +- Project [key#7, value#8, topic#9, partition#10, offset#11L, timestamp#12, timestampType#13, cast(value#8 as string) AS decoded_value#457]\n      +- StreamingExecutionRelation KafkaV2[Subscribe[trading4]], [key#7, value#8, topic#9, partition#10, offset#11L, timestamp#12, timestampType#13]\n'

In [3]:
# Decode the value column using UTF-8 encoding
decoded_df = df.withColumn("decoded_value", df["value"].cast("string").alias("decoded_value"))
# Parse the JSON data in the decoded_value column
parsed_df = decoded_df.selectExpr("CAST(decoded_value AS STRING)")


# Lưu kết quả vào bộ nhớ tạm
query = (parsed_df.writeStream
         .outputMode("append")
         .foreachBatch(lambda batchDF, batchId: batchDF.write.mode("append").saveAsTable("json"))
         .start())

# Chờ cho quá trình stream kết thúc
query.awaitTermination()

KeyboardInterrupt: 

In [ ]:
# Hiển thị dữ liệu từ bộ nhớ tạm
resultDF = spark.table("json")
resultDF.show()

In [60]:
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, FloatType

inner_schema = StructType([
    StructField("Close", FloatType(), True),
    StructField("Volume", FloatType(), True),
    StructField("Number of trades", FloatType(), True),
    StructField("Symbol", StringType(), True)
])

middle_schema = ArrayType(inner_schema)

outer_schema = ArrayType(middle_schema)


In [64]:
resultDF.

['decoded_value']

In [63]:
parsed_df = resultDF.select(from_json(resultDF.decoded_value, middle_schema).alias("data"))
parsed_df.show(truncate=False)


+----+
|data|
+----+
|null|
|null|
|null|
|null|
|null|
|null|
|null|
|null|
|null|
|null|
+----+



In [45]:
resultDF.collect()

[Row(decoded_value='[[[30305.6, 3.863, 292.0, 1.662, 0], [1937.88, 103.218, 178.0, 47.895, 1], [250.2, 657.459, 176.0, 320.378, 2], [0.718, 192537.0, 236.0, 73573.0, 3], [95.07, 305.664, 51.0, 29.955, 4]], [[30305.61, 4.362, 301.0, 1.992, 0], [1937.88, 103.218, 178.0, 47.895, 1], [250.2, 659.328, 180.0, 320.878, 2], [0.718, 192751.0, 239.0, 73590.0, 3], [95.07, 306.583, 53.0, 29.955, 4]], [[30305.61, 4.536, 304.0, 2.157, 0], [1937.89, 103.318, 179.0, 47.995, 1], [250.2, 660.305, 185.0, 321.229, 2], [0.718, 195594.0, 242.0, 76433.0, 3], [95.08, 306.688, 54.0, 30.06, 4]], [[30305.61, 4.626, 316.0, 2.244, 0], [1937.88, 104.691, 183.0, 48.368, 1], [250.2, 661.179, 190.0, 321.752, 2], [0.717, 210806.0, 252.0, 89008.0, 3], [95.08, 313.629, 59.0, 30.739, 4]], [[30305.61, 4.696, 325.0, 2.308, 0], [1937.88, 104.765, 186.0, 48.368, 1], [250.2, 666.016, 197.0, 323.887, 2], [0.718, 240661.0, 270.0, 118863.0, 3], [95.08, 313.629, 59.0, 30.739, 4]]]'),
 Row(decoded_value='[[[30305.6, 3.775, 284.0, 1

In [48]:
parsed_df.show()

+----+
|data|
+----+
|null|
|null|
|null|
|null|
|null|
|null|
|null|
|null|
|null|
|null|
+----+



In [38]:
resultDF.select("decoded_value").show(truncate=False)


+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|decoded_value                                                      

In [7]:
data[1].decoded_value

'[[[30305.6, 3.775, 284.0, 1.591, 0], [1937.89, 103.096, 177.0, 47.895, 1], [250.3, 655.191, 173.0, 320.378, 2], [0.718, 183152.0, 230.0, 68443.0, 3], [95.07, 305.664, 51.0, 29.955, 4]], [[30305.6, 3.863, 292.0, 1.662, 0], [1937.88, 103.218, 178.0, 47.895, 1], [250.2, 657.459, 176.0, 320.378, 2], [0.718, 192537.0, 236.0, 73573.0, 3], [95.07, 305.664, 51.0, 29.955, 4]], [[30305.61, 4.362, 301.0, 1.992, 0], [1937.88, 103.218, 178.0, 47.895, 1], [250.2, 659.328, 180.0, 320.878, 2], [0.718, 192751.0, 239.0, 73590.0, 3], [95.07, 306.583, 53.0, 29.955, 4]], [[30305.61, 4.536, 304.0, 2.157, 0], [1937.89, 103.318, 179.0, 47.995, 1], [250.2, 660.305, 185.0, 321.229, 2], [0.718, 195594.0, 242.0, 76433.0, 3], [95.08, 306.688, 54.0, 30.06, 4]], [[30305.61, 4.626, 316.0, 2.244, 0], [1937.88, 104.691, 183.0, 48.368, 1], [250.2, 661.179, 190.0, 321.752, 2], [0.717, 210806.0, 252.0, 89008.0, 3], [95.08, 313.629, 59.0, 30.739, 4]]]'

In [9]:
from pyspark.sql.functions import from_json
from pyspark.sql.types import StructType, StructField, FloatType, ArrayType
schema = StructType([
    StructField("data", ArrayType(ArrayType(FloatType())), nullable=False)
])


In [10]:
df_decode =resultDF.select(from_json(df.value, schema).alias("data"))

AnalysisException: "cannot resolve 'jsontostructs(`value`)' due to data type mismatch: argument 1 requires string type, however, '`value`' is of binary type.;;\n'Project [jsontostructs(StructField(data,ArrayType(ArrayType(FloatType,true),true),false), value#8, Some(Etc/UTC)) AS data#455]\n+- SubqueryAlias `default`.`json`\n   +- Relation[decoded_value#449] parquet\n"

In [13]:
from pyspark.sql.functions import col,from_json
newDF = resultDF.select(from_json(col("decoded_value"), schema)).withColumnRenamed("jsontostructs(decoded_value)", "value")
newDF.show()

+--------------------+
|               value|
+--------------------+
|[[1689405120000, ...|
|[[1689405120000, ...|
|[[1689405120000, ...|
|[[1689405120000, ...|
+--------------------+



In [15]:
from pyspark.sql.functions import col,from_json
newDF = resultDF.withColumn("decoded_value", from_json(col("decoded_value"), schema))
                

In [16]:
import numpy as np

def f(x): print(np.array(x.decoded_value).reshape(-1))
newDF.foreach(f)

In [19]:
type(resultDF)

pyspark.sql.dataframe.DataFrame

In [57]:
import json
# list_data = [np.frombuffer(s.value) for s in data]
list_data = [json.loads(s.value.decode()) for s in data]

In [66]:
list_data[1]

{'close': {'1682899260000': 1877.08},
 'volume': {'1682899260000': 423.2665},
 'count': {'1682899260000': 477},
 'taker_buy_volume': {'1682899260000': 262.549},
 'symbol': {'1682899260000': 1}}

In [64]:
list_data[1]['close']['1682899260000']

1877.08

In [41]:
np.frombuffer(data[-1].value, dtype=np.float32)

array([-3.0890025e-27,  2.3674374e+00, -5.0048828e-05,  4.3277731e+00,
        0.0000000e+00,  2.9375000e+00,  1.2666203e-26,  3.6054492e+00,
        0.0000000e+00,  2.2500000e+00,  2.7200830e+23,  3.8290038e+00,
        2.9900800e+02,  3.7568691e+00,  0.0000000e+00,  3.6582031e+00,
        1.7935223e-36,  3.3329062e+00,  0.0000000e+00,  2.0000000e+00,
        1.2520206e-24,  1.6978500e+00, -1.5881868e-23,  6.7989163e+00,
        0.0000000e+00,  3.0156250e+00,  0.0000000e+00,  5.9477539e+00,
        0.0000000e+00,  0.0000000e+00, -2.3314683e-17,  6.8918915e+00,
       -4.5420889e+37,  3.0196970e+00,  0.0000000e+00,  3.9853516e+00,
        5.2545659e+10,  2.6865005e+00,  0.0000000e+00,  2.1250000e+00,
        1.9023206e+17,  4.9153514e+00,  1.2279345e+12,  3.6108675e+00,
        0.0000000e+00,  3.7031250e+00,  4.0564818e+29,  3.4022362e+00,
        0.0000000e+00,  1.8750000e+00], dtype=float32)

In [13]:
data = resultDF.collect()

# # Extract the 'content' column and convert it to a NumPy array
# content_array = np.array([(row.value) for row in data])

In [14]:
data

[Row(decoded_value='[{"Open time": 1689150600000, "Close": "30720.15000000", "Volume": "31.63150000", "Number of trades": 1028, "Taker buy base asset volume": "24.43206000", "Symbol": "BTCUSDT"}, {"Open time": 1689150600000, "Close": "1888.03000000", "Volume": "230.96050000", "Number of trades": 380, "Taker buy base asset volume": "57.35300000", "Symbol": "ETHUSDT"}, {"Open time": 1689150600000, "Close": "248.60000000", "Volume": "1438.97500000", "Number of trades": 436, "Taker buy base asset volume": "877.79900000", "Symbol": "BNBUSDT"}, {"Open time": 1689150600000, "Close": "0.47420000", "Volume": "857225.00000000", "Number of trades": 411, "Taker buy base asset volume": "436428.00000000", "Symbol": "XRPUSDT"}, {"Open time": 1689150600000, "Close": "97.06000000", "Volume": "215.17200000", "Number of trades": 87, "Taker buy base asset volume": "82.88900000", "Symbol": "LTCUSDT"}]'),
 Row(decoded_value='[{"Open time": 1689150600000, "Close": "30720.16000000", "Volume": "31.31150000", "

In [48]:
resultDF.show()

+--------------------+
|               value|
+--------------------+
|[66 66 66 66 66 1...|
|[66 66 66 66 66 1...|
|[66 66 66 66 66 1...|
|[66 66 66 66 66 1...|
|[66 66 66 66 66 1...|
|[66 66 66 66 66 1...|
|[66 66 66 66 66 1...|
|[66 66 66 66 66 1...|
|[1D 5A 64 3B DF 4...|
|[1D 5A 64 3B DF 4...|
|[1D 5A 64 3B DF 4...|
|[1D 5A 64 3B DF 4...|
|[1D 5A 64 3B DF 4...|
|[1D 5A 64 3B DF 4...|
|[1D 5A 64 3B DF 4...|
|[1D 5A 64 3B DF 4...|
|[1D 5A 64 3B DF 4...|
|[1D 5A 64 3B DF 4...|
|[1D 5A 64 3B DF 4...|
|[1D 5A 64 3B DF 4...|
+--------------------+
only showing top 20 rows



In [24]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Hàm UDF để chuyển đổi mảng hex thành chuỗi
hex_to_string = udf(lambda arr: ''.join([chr(int(x, 16)) for x in arr]), StringType())

# Áp dụng hàm UDF để chuyển đổi cột "value"
df_processed = df.withColumn("processed_value", hex_to_string(resultDF.value))

# Hiển thị dữ liệu đã xử lý
df_processed.show()


AnalysisException: 'Resolved attribute(s) value#1511 missing from timestampType#97,partition#94,value#92,offset#95L,timestamp#96,key#91,topic#93 in operator !Project [key#91, value#92, topic#93, partition#94, offset#95L, timestamp#96, timestampType#97, <lambda>(value#1511) AS processed_value#1517]. Attribute(s) with the same name appear in the operation: value. Please check if the right attribute(s) are used.;;\n!Project [key#91, value#92, topic#93, partition#94, offset#95L, timestamp#96, timestampType#97, <lambda>(value#1511) AS processed_value#1517]\n+- StreamingRelationV2 org.apache.spark.sql.kafka010.KafkaSourceProvider@4878766f, kafka, Map(subscribe -> trading2, kafka.bootstrap.servers -> kafka1:19091,kafka2:19092,kafka3:19093), [key#91, value#92, topic#93, partition#94, offset#95L, timestamp#96, timestampType#97], StreamingRelation DataSource(org.apache.spark.sql.SparkSession@1e381879,kafka,List(),None,List(),None,Map(subscribe -> trading2, kafka.bootstrap.servers -> kafka1:19091

In [11]:
from pyspark.sql.functions import udf
from pyspark.sql.types import BinaryType

In [13]:

# Định nghĩa hàm giải mã
def decode_value(value):
    bytes_data = bytes.fromhex(value)
    numpy_array = np.frombuffer(bytes_data, dtype=np.float64)
    return numpy_array.tolist()

# Đăng ký UDF (User-Defined Function)
decode_udf = udf(decode_value, BinaryType())

# Lựa chọn cột "value" và áp dụng UDF để giải mã
decoded_df = df.select(decode_udf(df.value).alias("decoded_value"))

# Lưu trữ kết quả vào bộ nhớ
query = (decoded_df.writeStream
         .outputMode("append")
         .format("memory")
         .queryName("decoded_values")
         .start())

# Chờ cho quá trình stream kết thúc
query.awaitTermination()

StreamingQueryException: 'Job aborted due to stage failure: Task 0 in stage 29.0 failed 1 times, most recent failure: Lost task 0.0 in stage 29.0 (TID 29, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):\n  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main\n    process()\n  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process\n    serializer.dump_stream(func(split_index, iterator), outfile)\n  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 352, in dump_stream\n    self.serializer.dump_stream(self._batched(iterator), stream)\n  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 142, in dump_stream\n    for obj in iterator:\n  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 341, in _batched\n    for item in iterator:\n  File "<string>", line 1, in <lambda>\n  File "/usr/local/spark

In [8]:
result = spark.sql("SELECT * FROM decoded_values")
result.show()

AnalysisException: 'Table or view not found: decoded_values; line 1 pos 14'